Mount drive

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import packages

In [48]:
import pandas as pd
import csv
from collections import defaultdict

Read in files

In [49]:
survey = "nat"
nat_filler_path = "/content/drive/MyDrive/projects/2023/glass_project/5_9_23/naturalness_fillers_050923.csv"
typ_filler_path = "/content/drive/MyDrive/projects/2023/glass_project/5_9_23/typicality_fillers_050923.csv"
filler_path = nat_filler_path if survey == "nat" else typ_filler_path
df_filler = pd.read_csv(filler_path)
df_filler

,verb,sentence,zscore,order,block
0,arranged,The baby arranged the something.,-0.228202,1,calibration
1,split,The cow split the check.,0.641458,8,calibration
2,bounced,The player bounced the bottle.,0.866261,15,calibration
3,noticed,The stylist noticed the abruptly.,-0.869346,16,calibration
4,drove,The chauffeur drove the carefully.,-0.927087,26,calibration
5,shrunk,The washer shrunk the laundry.,1.048023,36,calibration
6,bumped,The car bumped the curb.,1.062882,41,calibration
7,clipped,The gardener clipped the was.,-1.069118,44,calibration
8,arranged,The time arranged the blocks.,0.149336,0,experiment
9,scratched,The dog scratched the war.,0.262877,2,experiment


### Now want to read in some "sampled" sentences (i.e., from ChatGPT) and then make some csv files for our experiment, each with ~60 target items and ~40 calibration items for a total of exactly 100.

In [50]:
# sampled = pd.read_csv("/content/drive/MyDrive/projects/2023/glass_project/01_19_24/llama_llamasenses_generation_top_four.csv")
sampled = pd.read_csv("/content/drive/MyDrive/projects/2023/glass_project/01_26_24/llama_propbanksenses_generation_top_four.csv")
# sampled = pd.read_csv("/content/drive/MyDrive/projects/2023/glass_project/09_15_23/stephanie_sentences_with_sense.csv")

Leaving out the profanity check because it deleted "lube" and "ovary".

In [51]:
# !pip install better_profanity

In [52]:
# def contains_profanity(sentence):
#   return profanity.contains_profanity(sentence)

# # Create a list to store sentences with profanity
# profanity_sentences = []

# # Iterate through the DataFrame and identify sentences with profanity
# for sentence in sampled["constructed_sent"]:
#     if contains_profanity(sentence):
#         profanity_sentences.append(sentence)

# # Print sentences with profanity
# for sentence in profanity_sentences:
#     print("Profanity found in sentence:", sentence)

# # Remove sentences with profanity from the DataFrame
# sampled = sampled[~sampled["constructed_sent"].apply(lambda x: contains_profanity(x))]

# print("Number of rows removed:", len(profanity_sentences))
# sampled

In [53]:
import math

num_lists = math.ceil(len(sampled) / 60)

In [54]:
print(num_lists)

13


In [55]:
import numpy as np
np.random.seed(0)

sampled["list_number"] = np.random.randint(1,num_lists + 1, size=len(sampled))


In [56]:
sampled["list_number"].value_counts()

4     82
9     64
10    62
6     61
5     61
1     60
12    60
3     59
8     56
2     55
7     54
13    51
11    51
Name: list_number, dtype: int64

Want to make sure no more than 60 with a given "list number."

In [57]:
import random
random.seed(0)


def no_more_than_n(df, max_number):
  max_number = max_number-1
  available_numbers = set(df["list_number"].unique())
  counts_dict = defaultdict(int)
  # keys are list numbers, values are number of rows assigned to that list.
  for index, row in df.iterrows():
    list_number = row["list_number"]
    if counts_dict[list_number] >= max_number:
      if list_number in available_numbers:
        available_numbers.remove(list_number)
      if available_numbers:
        new_list_number = random.choice(list(available_numbers))
        df.at[index, "list_number"] = new_list_number # updates row.
        counts_dict[new_list_number] += 1
      elif not available_numbers:
        print("Error, all list numbers are maxed out!")
    elif counts_dict[list_number] < max_number:
      counts_dict[list_number] += 1
  return df

In [58]:
sampled = no_more_than_n(sampled, 60)

In [59]:
sampled["list_number"].value_counts()

2     67
7     60
13    59
6     59
1     59
4     59
12    59
8     59
10    59
3     59
5     59
9     59
11    59
Name: list_number, dtype: int64

Now want to go through and create separate lists for our experiment.  Each list should have all 40 fillers plus about 60 target items, plus a few extras to get to exactly 100 total.

In [60]:
sampled = sampled[["verb", "sense", "sentence", "list_number"]]
# sampled = sampled[["verb_lemma", "sense", "constructed_sent", "list_number"]]
new_columns = {'verb_lemma': 'verb', 'constructed_sent': 'sentence'}
sampled = sampled.rename(columns=new_columns)


In [61]:
sampled["item_type"] = "target"

In [62]:
# calib = calib.rename(columns = {"Verb": "verb"})
calib = df_filler
calib = calib[calib["block"] != "calibration"]


If there's less than 100 in a given list, we add a few "extras" until it has 100 items in it.

In [63]:
def create_stim_csvs(calib, sampled):
  num_lists = sampled["list_number"].max()
  filler_df = calib[["verb", "sentence", "order"]]
  filler_df["item_type"] = "filler"
  filler_df["sense"] = "n/a"
  filler_df["list_number"] = "all"
  filler_df = filler_df.sort_values("order")
  # extras["sense"] = "n/a"
  for i in range(1, num_lists+1):
    target_df_subset = sampled.loc[sampled["list_number"] == i]
    extras_needed = 89-len(target_df_subset)
    extra_fillers = filler_df.head(extras_needed)
    combined = pd.concat([target_df_subset, extra_fillers])
    combined
    combined.to_csv(str(survey) + "_" + str(i) + ".csv")
    # combined = pd.concat([filler_df, target_df_subset])
    # extras_needed = 92 - len(combined)
    # extra_fillers = extras2.head(extras_needed)
    # extra_fillers["list_number"] = "extra"
    # extra_fillers["item_type"] = "extra"
    # print(combined.shape)
    # combined = pd.concat([combined, extra_fillers])
    # combined.to_csv(str(survey) + "_" + str(i) + ".csv")

In [64]:
create_stim_csvs(calib, sampled)


<ipython-input-63-233435e95cc1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filler_df["item_type"] = "filler"
